<a href="https://colab.research.google.com/github/jjennings955/CSE5368-Spring-2019/blob/master/Computational_Graphs_and_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Graphs
We can create a computational graph for any mathematical function

For example:
$f(x,y,z) = 2 x^2 cos(yz)$
<pre>












</pre>

or 
$f(x; W, b) = \sigma(Wx + b)$
<pre>












</pre>

We call $x, y, z$ **variables**, and the functions we apply to those variables **operations** (or functions). Variables and functions together we will call **nodes** in the graph.

- **variables** - can be scalars, vectors, matrices, tensors, or any mathematical object (but we'll stick to those in this course)
- **operations** can be any functions of those mathematical objects (so functions mapping scalars -> scalars, vectors -> vectors, or any other combination)

Some libraries may add further classifications of **nodes** (such as 'placeholders' in tensorflow), but those are implementation details (important to tensorflow, but not important to the concept of a computational graph)

Together these nodes and edges form a **directed acyclic graph** (or **DAG**), and we call this graph a computational graph.

## Forward Propagation
Forward propagation is the process of iterating over the graph, feeding values into variable, and evaluating each operation, collecting the result, and feeding it into any remaining operations down stream, until we have fully evaluated the graph and computed the final output. It may also be called **feed forward** or **the forward pass**.

We go through the graph in **topological order**. That is, we start by processing a node that has no predecessor in the graph, mark it as 'complete', and only process a node if all its predecessors have already been processed. A graph may have many valid topological orders.

This is intuitively what we do when we evaluate a mathematical expression without necessarily realizing it.

# Backpropagation
Backward propagation is a process we implement to compute gradients in computational graphs. Because in deep learning we use **gradient based optimization**, a method for efficiently and automatically computing the gradient of a complex function is something desirable. 

Backpropagation in other fields may be called **reverse mode automatic differentiation**.

In backward propagation, we iterate over the graph in **reverse topological order**, compute gradients, and feed those gradients backwards using **the chain rule**. You can think of backpropagation as simply a method for efficiently implementing the chain rule in multi-dimensions, for potentially very deeply nested chains of functions.

## The Chain Rule
In a single dimension, as in Calculus 101, the chain rule is simply as follows:
Given:

$y = f(x)$ and $x = g(t)$

$$\frac{dy}{dt} = \frac{dy}{dx} \frac{dx}{dt}$$

In multi-dimensions (but still with scalar valued functions), it gets slightly more complex, because chains of functions may not be completely linear (like in the examples we've seen).

Given $u = f(x,y)$ where $x = f_1(r,t)$ and $y = f_2(r,t)$

$$\frac{\delta u}{\delta t} = \frac{\delta u}{\delta x} \frac{\delta x}{\delta t} + \frac{\delta u}{\delta y} \frac{\delta y }{\delta t}$$

In english, as we go backwards in a computational graph, we **multiply along linear chains**, and **add where two chains converge**.
<pre>








</pre>


## Backpropagation Motivation
Rather than writing down a mathematical expression, symbolically computing an abstract formula for its gradient using calculus, and attempting to simplify it into something elegant, then implementing it into code, we can implement a library of general purpose operations, we just need to be able to define their **forward** behavior and their **backward** behavior.

The forward behavior simply computes the output of the function (as we've seen), the backward behavior computes the gradient of the output with respect to its inputs.

```python
class Operation:
  def forward(self, x):
    return f(x)
  def backward(self, g):
    return gradient_f(x)*g
 ```


## Vector Chain Rule
Generally speaking for deep learning, most of our functions will be vector -> vector (or even tensor->tensor) functions. The rules for these can be derived from vector->scalar functions, but we'll skip the proof:

$f : R^n \rightarrow R^m$ \\

You can think of a vector to vector function as $m$ functions of the vector $x$, stacked into a vector.

$ f(\textbf{x}) = \begin{bmatrix} f_1(\textbf{x}) \\ f_2(\textbf{x}) \\ ... \\ f_m(\textbf{x})\end{bmatrix}$


The Jacobian of f is defined as:

$\textbf{J}_f = \begin{bmatrix}{\dfrac {\partial f_{1}}{\partial x_{1}}}&\cdots &{\dfrac {\partial f_{1}}{\partial x_{n}}}\\\vdots &\ddots &\vdots \\{\dfrac {\partial f_{m}}{\partial x_{1}}}&\cdots &{\dfrac {\partial f_{m}}{\partial x_{n}}}\end{bmatrix}$

It can be seen that row $i$ of the Jacobian is the gradient (transposed) of $f_i(x)$ with respect to $\textbf{x}$

We introduce this rule, because it is used in the most general form of the chain rule (so far).

Given: $y = g(x) = \begin{bmatrix} g_1(x) \\ … \\ g_k(x) \end{bmatrix}$ and $z = f(y) = \begin{bmatrix} f_1(y) \\ … \\ g_k(y) \end{bmatrix}$ 

Then the gradient of $z$ with respect to x, is given by the following formula:

$\nabla_{x} (z) = J_{f}(x)^T \nabla_{y} (z)$

That is, to obtain the gradient of z with respect to x (through y), we multiply the Jacobian of y (transposed) by the gradient of z.

The following formula (without transposing the Jacobian) is equivalent:
$\nabla_{x} (z) = \nabla_{y} (z) J_{f}(x) $

## Vector Backpropagation
Now, just like with the scalar chain rule, we have our rules for the backward pass.

**Forward:**
- Draw/create computational graph
- Do forward propagation with given values, label computational graph with output
- Compute Jacobians at each operation, for each input variable

**Backward**
- Start with an initial value for the gradient of the output, a vector of 1's with the same dimension as our function's final output, or a scalar 1 if the output is scalar.
For each operation we have: a set of variables with their input values, and Jacobians from the forward pass.
- Move backwards in the graph (in reverse topological order), multiply the Jacobian transposed by the gradient coming backwards at each operation.
- We still add up the gradient if we have multiple branches that converge at the same variable.



## Example:
Given: 
$z = Wx$

$q = z^2$

and $W = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}$, $x = \begin{bmatrix} 0.5 \\ 1.2 \end{bmatrix}$
![Example 1](https://raw.githubusercontent.com/jjennings955/CSE5368-Spring-2019/master/assets/example_1_vector.svg?sanitize=true)
<pre>











</pre>

We start with an initial gradient of $\begin{bmatrix}1 \\ 1\end{bmatrix}$, and multiply by the Jacobian transposed at each step.

**Observation:** To compute the Jacobian of a function of a matrix, we end up 'unrolling' the matrix into a vector. We have to 'roll it back up' if we wish to get the dimensions of our gradient to match (for updating the weights of a matrix, for example).

## Generality of this approach
The Jacobian of a scalar-valued function, with respect to one of its scalar arguments is just a [1x1] matrix containing the partial derivative. So we can still consider the scalar examples done previously as 'multiplying the jacobian transposed times the gradient', it just reduces to scalar multiplication (Note: in general a 1x1 matrix **is not** the same as a scalar, you can't multiply a 1x1 matrix by a 2x2 matrix, for example)

We can easily construct the equivalent computational graph for the case of scalar valued functions, as shown below:

$z = Wx$

$q = z^2$

and $W = \begin{bmatrix} w_1 & w_2 \\ w_3 & w_4 \end{bmatrix}$, $x = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$

![Example 2](https://raw.githubusercontent.com/jjennings955/CSE5368-Spring-2019/master/assets/example_2_scalar.svg?sanitize=true)

<pre>











</pre>



# The Problem with Jacobians
Conceptually, the definition of backpropagation using Jacobians given above is exactly correct. However, practically, it has a lot of drawbacks.

## Jacobians are enormous, often sparse
For example, consider a simple vector function that performs elementwise squaring of a vector (like in our example above).

$f(\textbf{x}) = \begin{bmatrix} x_1^2 \\ x_2^2 \\ ... \\ x_n^2 \end{bmatrix}$

The resulting Jacobian will always be a square $[n \times n]$ matrix with 0's everywhere but the diagonal.

$J_f(x) =
  \begin{bmatrix}
    2x_{1}  \\
    & 2x_{2} \\
    & &  \ddots \\
    & & & 2x_{i} \\
    & & & &\ddots \\
    & & & && 2x_{n} \\
  \end{bmatrix}
$

## Jacobian-Vector Products
Because we actually don't care about the Jacobian itself, we only care about the Jacobian transposed times a vector, we can find a special formula for this **Jacobian-vector product** (jvp) that **doesn't require computing the Jacobian itself**

In the example above, the **Jacobian-vector product** is simply: $$J_f(x)^T \textbf{v} = 2 \textbf{x} \odot \textbf{v} = \begin{bmatrix} 2 x_1 v_1 \\ 2 x_2 v_2 \\ ... \\ 2 x_n v_n \end{bmatrix}$$

Where $a \odot b$ is the elementwise product ('Hadamard product" between a and b)


Takeaways:
- We can almost always find a quite compact expression for doing the **Jacobian-vector product**, eliminating the need to compute and store the Jacobian explicitly. We can always fall back to computing the Jacobian if it's not too expensive.
- Using these rules, we can often (but not always) avoid the annoying bookkeeping involved in reshaping our gradients back into matrices, shown in the numerical example.
- It only takes a small handful of these functions to implement most common neural network architectures, but libraries like tensorflow implement thousands of them for practically every function imaginable.
- A sub-optimal, but perhaps situationally workable alternative solution would be to use a **sparse matrix** package for storing the Jacobians and doing the products.

## Jacobian-free Backpropagation Algorithm
**Forward:**
- Draw/create computational graph
- Do forward propagation with given values, label computational graph with output


**Backward**
- Start with an initial value for the gradient of the output, a vector of 1's with the same dimension as our function's final output, or a scalar 1 if the output is scalar.
For each operation we have: a set of variables with their input values, and Jacobians from the forward pass.
- Move backwards in the graph (in reverse topological order), compute the **Jacobian-vector product** for the gradient coming backwards at each operation.
- We still add up the gradient if we have multiple branches that converge at the same variable.

```python
class Operation:
  def forward(self, x, y, z):
    self.x = x # cache values of inputs
    self.y = y
    self.z = z
    
    self.out = f(x, y, z)
    return self.out
  
  def backward(self, g):
    grad_x = jvp_x(g, x, y, z) # jvp_x implements the jacobian vector product between J_x and the gradient g
    grad_y = jvp_y(g, x, y , z) # jvp_y implements the jacobian vector product between J_y and the gradient g
    grad_z = jvp_z(g, x, y, z) # jvp_z implements the jacobian vector product between J_z and the gradient g
    return [grad_x, grad_y, grad_z] # an example of branching in the backwards flow
 ```

```python
class Square:
  def forward(self, x):
    self.x = x # cache values of inputs
    self.out = x**2 #elementwise_square
    return self.out
  
  def backward(self, g):
    return 2*self.x*g # compute jacobian vector product for the element-wise square function
 ```
